<a href="https://colab.research.google.com/github/iGhostlp/Albus/blob/Gunter-y-Ernesto/Proyecto_BBVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Armado del entorno

In [14]:
# Download Spark
!wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz

In [15]:
# Unzip the file
!tar xf spark-3.3.2-bin-hadoop3.tgz

In [16]:
!readlink -f $(which java) | sed "s:bin/java::"

/usr/lib/jvm/java-11-openjdk-amd64/


In [17]:
# Set up the environment for Spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64/"
os.environ["SPARK_HOME"] = '/content/spark-3.3.2-bin-hadoop3'

In [18]:
# Install library for finding Spark
!pip install -q findspark

# Import the libary
import findspark

# Initiate findspark
findspark.init()

In [19]:
# Import SparkSession
from pyspark.sql import SparkSession

# Create a Spark Session
spark = SparkSession.builder.master("local[*]").config('spark.sql.parquet.datetimeRebaseModeInRead','CORRECTED').getOrCreate()

# Check Spark Session Information
spark

# Extraccion de datos desde parquet, clientes y teléfonos.

In [37]:
df_customer = spark.read.load('sample_data/customer_basics.snappy.parquet', sep=',', inferschema='true', header='true')
df_phones = spark.read.load('sample_data/phones.snappy.parquet', sep=',', inferschema='true', header='true')


In [38]:
df_customer_phones = df_customer.join(df_phones, 'customer_id', how='inner')

In [39]:
df_customer_phones.show()

+-----------+-------------+-------------+--------------+---------------+--------------------+---------+----------------+-----------+-----------------------+-----------------------+----------+-------------------+-----------------+--------------------------+----------------+----------------+-------------------+-------------------+-----------+----------+----------------------+--------------------+---------------------+-----------+--------------------------+----------------------------+-------------------------------+------------------------+------------------------+--------------------------+------------------------+--------------------------+----------------------------+------------------+------------------------+-------------------+-------------------+---------------------------+-----------------+-------------------+-----------------+-------------------------------+--------------------+---------------------+------------------+----------------+--------------------+-----------------------

QUIERO: Filtrar el DataFrame de contactos telefónicos de clientes y resguardar los 3 contactos más actuales por cliente.   
PARA: Reducir el volumen de datos y trabajar solo con los más actualizados

registry_entry_date Momento en el que se realiza el alta de un registro
last_change_date Fecha en el que se registra en el sitema un cambio en la informacion

In [40]:
df_phones.sort('customer_id').show()

+-----------+--------------+-------------------+-----------------+----------+----------------+---------------+-------------+-------------------+-----------------+-------------+------------+---------------+-----------------------+------------------+---------------------+-----------------+----------+-----------+--------------------------+---------------------+--------------------+---------------------+----------------+-------------------------+-------------------------+------------------+-------------------+-----------------+--------------------+---------------------+-----------------------+----------------------+--------------------+----------------------+------------------------------+----------------------------+-------------------+----------------+----------------+-------------------+--------------------+-----------------------+---------------------+
|customer_id|phone_use_type|address_sequence_id|phone_sequence_id|phone_type|phone_country_id|prefix_phone_id|phone_area_id|cellphone_p

In [25]:
from pyspark.sql.functions import row_number, desc

In [26]:
from pyspark.sql.window import Window

In [55]:
df_phonesCut = df_phones.drop('phone_intern_id','phone_country_id', 'aditional_info_txt_desc', 'primary_phone_type','address_sequence_type','address_town_name','zipcode_id', 'province_id','sender_application_id','normalization_status_type','normalization_reason_name','validity_start_date','validity_end_date','dlvy_day_monday_type','dlvy_day_tuesday_type','dlvy_day_wednesday_type','dlvy_day_thursday_type','dlvy_day_friday_type','dlvy_day_friday_type','dlvy_day_saturday_type','delivery_contact_start_hm_date','delivery_contact_end_hm_date','operational_load_date','normalization_date')

In [63]:
df_phonesOrdenados = df_phonesCut.orderBy([df_phonesCut.customer_id, desc('last_change_date')])

df_phonesOrdenados.show()

+-----------+--------------+-------------------+-----------------+----------+---------------+-------------+-------------------+-----------------+-------------+------------+--------------------------+---------------------+--------------------+----------------+-------------------+----------------+----------------+-------------------+--------------------+-----------------------+
|customer_id|phone_use_type|address_sequence_id|phone_sequence_id|phone_type|prefix_phone_id|phone_area_id|cellphone_prefix_id|phone_exchange_id|phone_line_id|    phone_id|customer_phone_status_type|phone_status_mod_date|contact_channel_type|wrong_phone_type|registry_entry_date|register_user_id|last_change_date|last_change_user_id|last_change_hms_date|last_change_terminal_id|
+-----------+--------------+-------------------+-----------------+----------+---------------+-------------+-------------------+-----------------+-------------+------------+--------------------------+---------------------+--------------------+

In [64]:
import pyspark.sql.functions as f
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat,col

In [65]:
df_phonesOrdenados = df_phonesOrdenados.select(concat(df_phonesOrdenados.prefix_phone_id,df_phonesOrdenados.phone_area_id,df_phonesOrdenados.phone_exchange_id,df_phonesOrdenados.phone_line_id).alias('Full_Phone'),'customer_id','last_change_date')

In [69]:
df_phonesOrdenados.show()

+------------+-----------+----------------+
|  Full_Phone|customer_id|last_change_date|
+------------+-----------+----------------+
|542664697946|   00000007|      2022-09-01|
|543412847321|   00000039|      2022-09-20|
|541125064159|   00000044|      2022-10-03|
|543815909885|   00000381|      2022-07-19|
|542615904192|   00005527|      2022-10-03|
|542613862762|   00005527|      2022-10-03|
|541134340087|   00013438|      2022-09-08|
|541164891848|   00021991|      2022-08-09|
|541153115187|   00038236|      2022-08-10|
|541161577947|   00041884|      2022-07-28|
|541153376523|   00048126|      2022-10-24|
|541165187983|   00048225|      2022-07-08|
|541169951912|   00052103|      2022-09-27|
|542966425661|   00064339|      2022-07-07|
|543584112643|   00071569|      2022-11-25|
|542915740275|   00072996|      2022-11-23|
|542614372519|   00076782|      2022-08-03|
|541151522021|   00077558|      2022-06-29|
|542974779188|   00084439|      2022-10-20|
|542214218843|   00084439|      

In [87]:
df_phonesOrdenados.repartition(3).sortWithinPartitions('customer_id','Full_phone','last_change_date').withColumn("partition", f.col(spark_partition_id)).show();

NameError: ignored